In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()
%matplotlib inline

In [2]:
#%unload_ext vimception


In [63]:
press = np.array([1000,950,900,850,700,500,400,300,250,200,150,100,70,50,30,20,10])*100
nplevs= len(press)

In [11]:
height_match = np.load('height_match_N96L852000_hossaini_press.npy')            

In [10]:
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006]

def read_in_data(year):
    filename='/Users/ptg21/data/projects/TRANSCOM/N96_results/CH2Br2_'+str(year)+'.nc'
    ncfile1    = netCDF4.Dataset(filename)
    lon1        = ncfile1.variables['longitude'][:]
    lat1        = ncfile1.variables['latitude'][:] 
    times1      = ncfile1.variables['t']
    hybrid_ht1  = ncfile1.variables['hybrid_ht'][:]

    field_34062      = ncfile1.variables['field34062'][:,:,:,:] # liang = 62
    field_34063      = ncfile1.variables['field34063'][:,:,:,:] # ordonez = 63
    field_34065      = ncfile1.variables['field34065'][:,:,:,:] # ziska = 65

    nmonths, nlevels, ny1, nx1 = np.shape(field_34062)
    # return liang, ordonez, ziska + dims
    return field_34062, field_34063, field_34065, nmonths, nlevels, ny1, nx1

field_l, field_o, field_z, nmonths, nlevels1, ny1, nx1 = read_in_data(year)

In [17]:
def interp_lin_val(xnew, x, y, idx):
    # x,y arrays
    # xnew is the new xvalue
    # returns ynew as a linear interpolation
    # returns a value
    dx = xnew - x[idx]
    dy_dx = (y[idx]-y[idx+1])/(x[idx]-x[idx+1]) 
    # calculate a value at thew intermediate pressure
    return y[idx]+dy_dx*dx

In [18]:
liang_res   = np.empty([nmonths, nplevs, ny1, nx1])
ordonez_res = np.empty([nmonths, nplevs, ny1, nx1])
ziska_res   = np.empty([nmonths, nplevs, ny1, nx1])
# tried this using interp1d but was factor of 3 slower!  
years=[1998]
for year in years:
    print year
    field_l, field_o, field_z, nmonths, nlevels1, ny1, nx1 = read_in_data(year)

    for imonth in range (0, 12):
        print imonth
        for ilat in range(nyl, nyu):
            for ilon in range (nxl, nxu): 
                x_arr = pressure[imonth, :, ilat, ilon]
                for ilev in range(0, nplevs):
                    if height_match[imonth,ilev, ilat, ilon] != -999:
                            liang_res[imonth,  ilev, ilat, ilon] = interp_lin_val(press[ilev], 
                                                                                  x_arr, 
                                                                                  field_l[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])
                            ordonez_res[imonth,ilev, ilat, ilon] =  interp_lin_val(press[ilev], 
                                                                                  x_arr, 
                                                                                  field_o[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])
                            ziska_res[imonth,  ilev, ilat, ilon] = interp_lin_val(press[ilev], 
                                                                                  x_arr,
                                                                                  field_z[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])

                    else:
                        liang_res[imonth,  ilev, ilat, ilon] = -999
                        ordonez_res[imonth,ilev, ilat, ilon] = -999 
                        ziska_res[imonth,  ilev, ilat, ilon] = -999 


    np.save('ordonez_res'+str(year)+'.npy',ordonez_res)
    np.save('liang_res'+str(year)+'.npy',liang_res)
    np.save('ziska_res'+str(year)+'.npy',ziska_res)

1998
0
1
2
3
4
5
6
7
8
9
10
11
